In [2]:
import tensorflow as tf
import numpy as np
import deepxde as dde

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


No backend selected.
Finding available backend...
Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Instructions for updating:
non-resource variables are not supported in the long term



In [3]:
def residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2):
    
    f_u = u_t + l1*(u*u_x + v*u_y) + p_x - l2* (u_xx + u_yy)
    f_v = v_t + l1*(u*v_x + v*v_y) + p_y - l2* (v_xx + v_yy)

    return f_u, f_v

class PhysicsInformedNN:
    def __init__(self, lb, ub, layers, u0, v0, x0, X):

        self.lb = lb
        self.ub = ub
                
        self.layers = layers

        self.u0 = u0
        self.v0 = v0
        self.x0 = x0
        self.X = X

        self.model = self.initialize_NN(self,layers)

    def initialize_NN(self,layers):
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(layers[0]))
        scaling_layer = tf.keras.layers.Lambda(
            lambda x: 2.0*(x-self.lb)/(self.ub-self.lb)-1.0)
        model.add(scaling_layer)
        num_layers = len(layers)
        for i in range(1,num_layers-2):
            model.add(tf.keras.layers.Dense(layers[i],
                                            activation=tf.keras.activations.get('tanh'),
                                            kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.Dense(layers[-1]))

        return model
    

    def loss(self, X, X0, u0, v0):
        psi_and_p = self.model(X0)
        u_pred =  psi_and_p[:,0:1]
        v_pred =  psi_and_p[:,1:2]
        p_pred = psi_and_p[:,2:3]

        loss = tf.reduce_mean(tf.square(u0-u_pred))
        loss += tf.reduce_mean(tf.square(v0-v_pred)) #Revisar

        r1,r2 = self.get_residual(self,X)

        phi_ru = tf.reduce_mean(tf.square(r1))
        phi_rv = tf.reduce_mean(tf.square(r2))

        loss += phi_ru 
        loss += phi_rv

        return loss
    
    def get_residual(self,X):
        with tf.GradientTape(persistent=True) as tape:
            x = X[:,0:1]
            y = X[:,1:2]
            t = X[:,2:3]

            tape.watch(x)
            tape.watch(y)
            tape.watch(t)

            psi_and_p = self.model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))

            u = psi_and_p[:,0:1]
            v = psi_and_p[:,1:2]
            p = psi_and_p[:,2:3]


            u_t = tape.gradient(u,t)
            v_t = tape.gradient(v,t)

            u_x = tape.gradient(u,x)
            u_xx = tape.gradient(u_x,x)

            u_y = tape.gradient(u,y)
            u_yy = tape.gradient(u_y,y)

            v_x = tape.gradient(v,x)
            v_xx = tape.gradient(v_x,x)

            v_y = tape.gradient(v,y)
            v_yy = tape.gradient(v_y,y)

            p_x = tape.gradient(p,x)
            p_y = tape.gradient(p,y)

        del tape
        
        l1 = self.lambda1
        l2 = self.lambda2
        f_u, f_v = residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2)

        return f_u, f_v
    
    def loss_gradient(self,X,X0,u0, v0):

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.model.trainable_variables)
            loss = self.loss(self, X, X0, u0, v0)
            g = tape.gradient(loss, self.model.trainable_variables)

        del tape
        return loss, g

    def optimization(self, cor=50, tol=1.0  * np.finfo(float).eps,  iter=50000, fun=50000, ls=50):
        def time_step():
            loss = self.loss(self, self.X, self.X0, self.u0, self.v0)
            return loss
        variables = self.model.trainable_variables

        dde.optimizers.config.set_LBFGS_options(maxcor=cor, ftol=tol,  maxiter=iter, maxfun=fun, maxls=ls)
        dde.optimizers.tfp_optimizer.lbfgs_minimize(variables, time_step)


In [5]:
import scipy

In [8]:
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 3]
    
    # Load Data
data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')

U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

In [ ]:
idx = np.random.choice(N*T, 5000, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

# Training
model = PhysicsInformedNN(lb, ub, layers, u0, v0, x0, X):
model.train(200000)